In [232]:
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt
from matplotlib import cm
import multiprocessing as mp
from stitch import stitch

In [233]:
# Household parameters
life = 80
S = 80
working = round(9 * S / 16)
beta_annual = .96
beta = beta_annual ** (life / S)
    
sigma = 3.0

nvec_h = np.array([1.0] * working + [0.2] * (S - working))
nvec_f = np.array([1.0] * working + [0.2] * (S - working))

L_h = np.sum(nvec_h)
L_f = np.sum(nvec_f)

# Firm parameters
alpha_h = 0.4
alpha_f = 0.35

phi_h = 1
phi_f = 1

Z_h = 1.1
Z_f = 1.0

gamma_h = 0.4
gamma_f = 0.35

delta_annual_h = 0.06
delta_h = 1 - ((1 - delta_annual_h) ** (life / S))

delta_annual_f = 0.05
delta_f = 1 - ((1 - delta_annual_f) ** (life / S))

# SS parameters
SS_tol = 1e-13
xi = 0.6

In [234]:
@stitch(lambda cvec, sigma: -sigma / (cvec ** (1 + sigma)), (0, None), 1e-10)
def get_MUvec(cvec, sigma):
    return 1 / (cvec ** sigma)

@stitch(lambda x, gamma: gamma / (x ** (1 - gamma)), (0, None), 1e-10)
def root(x, gamma):
    return x ** gamma

def aggregate(r_1, r_2, q, alpha, phi):
    if phi==1:
        constant = ((1 - alpha) ** (1 - alpha)) * (alpha ** alpha)
        answer = (1 / constant) * root(r_1, (1 - alpha)) * root((q * r_2), alpha)
    else:
        constant = (1 - alpha) * root(r_1, (1 - phi)) + alpha * root((q * r_2), (1 - phi))
        answer = root(constant, 1 / (1 - phi))
    return answer

def get_K2L(r, Z, gamma, delta):
    return root((r + delta) / (gamma * Z), 1 / (gamma - 1))

def get_K(r, r_sub, share, ex, phi, K):
    return share * root((ex * r_sub / r), -phi) * K

def get_w(K2L, Z, gamma):
    return (1 - gamma) * Z * root(K2L, gamma)

def get_r(r, ex, alpha, K, K_sub, phi):
    return ex * r * root(alpha * K / K_sub, 1/ phi)
    
def get_Y(Z, K, L, gamma):
    return Z * root(K, gamma) * root(L, (1 - gamma))

In [235]:
def get_cvec(rvec, wvec, bvec, S, nvec, b_init=0):
    extended_bvec = np.zeros(S + 1)
    if b_init:
        extended_bvec[0] = b_init
    extended_bvec[1:-1] = bvec
    cvec = nvec * wvec + (1 + rvec) * extended_bvec[:-1] - extended_bvec[1:]
    return cvec

def bvec_error(bvec, *args):
    r, w, S, beta, sigma, nvec = args
    cvec = get_cvec(r, w, bvec, S, nvec)
    MUvec = get_MUvec(cvec, sigma)
    error = MUvec[:-1] - beta * (1 + r) * MUvec[1:]
    return error

In [236]:
def get_SS(HH_params, F_params, SS_params, guess):
    S, beta, sigma, nvec_h, nvec_f, L_h, L_f = HH_params
    Z_h, Z_f, gamma_h, gamma_f, delta_h, delta_f, alpha_h, alpha_f, phi_h, phi_f = F_params
    SS_tol, xi = SS_params
    q, r_h, r_f = guess
    
    iterate = 0
    error = 100
    
    while (error > SS_tol) & (iterate < 500): 
        r = aggregate(r_h, r_f, 1 / q, alpha_h, phi_h)
        r_star = aggregate(r_f, r_h, q, alpha_f, phi_f)

        K2L_h = get_K2L(r, Z_h, gamma_h, delta_h)
        w_h = get_w(K2L_h, Z_h, gamma_h)

        K2L_f = get_K2L(r_star, Z_f, gamma_f, delta_f)
        w_f = get_w(K2L_f, Z_f, gamma_f)

        # Solve for savings using Euler Equations
        bvec_h_guess = np.linspace(1, S - 1, num = S - 1)
        bvec_h_args = (r_h, w_h, S, beta, sigma, nvec_h)
        bvec_h_result = opt.root(bvec_error, bvec_h_guess, args=bvec_h_args, tol=SS_tol)
        if not bvec_h_result.success:
            print("Root finder for bvec_h failed on iteration {}".format(iterate))
            bvec_h = bvec_h_result.x
        else:
            bvec_h = bvec_h_result.x

        bvec_f_guess = np.linspace(1, S - 1, num = S - 1)
        bvec_f_args = (r_f, w_f, S, beta, sigma, nvec_f)
        bvec_f_result = opt.root(bvec_error, bvec_f_guess, args=bvec_f_args, tol=SS_tol)
        if not bvec_f_result.success:
            print("Root finder for bvec_f failed on iteration {}".format(iterate))
            bvec_f = bvec_f_result.x
        else:
            bvec_f = bvec_f_result.x

        cvec_h = get_cvec(r_h, w_h, bvec_h, S, nvec_h)
        cvec_f = get_cvec(r_f, w_f, bvec_f, S, nvec_f)

        plot=False
        if plot:
            plt.figure(figsize=(12, 9))
            plt.plot(bvec_h)
            plt.plot(bvec_f)
            plt.show()

        # Calculate the capital values
        K_h = L_h * K2L_h
        K_f = L_f * K2L_f

        Y_h = get_Y(Z_h, K_h, L_h, gamma_h)
        Y_f = get_Y(Z_f, K_f, L_f, gamma_f)

        K_hh = get_K(r, r_h, 1 - alpha_h, 1, phi_h, K_h)
        K_ff = get_K(r_star, r_f, 1 - alpha_f, 1 , phi_f, K_f)

        K_fh = np.sum(bvec_h) - K_hh
        K_hf = np.sum(bvec_f) - K_ff

        # Update guesses of r_h, r_f, q
        r_h_new = get_r(r_star, 1 / q, alpha_f, K_f, K_fh, phi_f)
        r_f_new = get_r(r, q, alpha_h, K_h, K_hf, phi_h)
        q_new = (r_f * K_hf) / (r_h * K_fh)
        
        error = (r_h_new - r_h) ** 2 + (r_f_new - r_f) ** 2 + (q_new - q) ** 2
        r_h = xi * r_h_new + (1 - xi) * r_h
        r_f = xi * r_f_new + (1 - xi) * r_f
        q = xi * q_new + (1 - xi) * q
        
        iterate = iterate + 1
        print("Outer Loop Iteration Number: ", iterate)
    if iterate==500:
        print("Did not converge. Error: ", error)
    else:
        print("Converged after iteration: ", iterate, " Error: ", error)
    print("R_h, R_f, q: ", r_h, r_f, q)
    return (r_h, r_f, q, cvec_h, cvec_f, bvec_h, bvec_f)

In [237]:
HH_params = (S, beta, sigma, nvec_h, nvec_f, L_h, L_f)
F_params = (Z_h, Z_f, gamma_h, gamma_f, delta_h, delta_f, alpha_h, alpha_f, phi_h, phi_f)
SS_params = (SS_tol, xi)
guess = (0.05, 0.06, 1)
r_h, r_f, q, cvec_h, cvec_f, bvec_h, bvec_f = get_SS(HH_params, F_params, SS_params, guess)

Root finder for bvec_h failed on iteration 0
Root finder for bvec_f failed on iteration 0
Outer Loop Iteration Number:  1
Root finder for bvec_h failed on iteration 1
Root finder for bvec_f failed on iteration 1
Outer Loop Iteration Number:  2
Root finder for bvec_h failed on iteration 2
Root finder for bvec_f failed on iteration 2
Outer Loop Iteration Number:  3
Root finder for bvec_h failed on iteration 3
Root finder for bvec_f failed on iteration 3
Outer Loop Iteration Number:  4
Root finder for bvec_h failed on iteration 4
Root finder for bvec_f failed on iteration 4
Outer Loop Iteration Number:  5
Root finder for bvec_h failed on iteration 5
Root finder for bvec_f failed on iteration 5
Outer Loop Iteration Number:  6
Root finder for bvec_h failed on iteration 6
Root finder for bvec_f failed on iteration 6
Outer Loop Iteration Number:  7
Root finder for bvec_h failed on iteration 7
Root finder for bvec_f failed on iteration 7
Outer Loop Iteration Number:  8
Root finder for bvec_h f

/Users/rubyzhang/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: overflow encountered in power
  app.launch_new_instance()
/Users/rubyzhang/Desktop/OLG/LgOpenOG/stitch.py:16: RuntimeWarning: invalid value encountered in less
  mask = x < lb


Root finder for bvec_f failed on iteration 42
Outer Loop Iteration Number:  43
Root finder for bvec_h failed on iteration 43
Root finder for bvec_f failed on iteration 43
Outer Loop Iteration Number:  44
Root finder for bvec_h failed on iteration 44
Root finder for bvec_f failed on iteration 44
Outer Loop Iteration Number:  45
Root finder for bvec_h failed on iteration 45
Root finder for bvec_f failed on iteration 45
Outer Loop Iteration Number:  46
Root finder for bvec_h failed on iteration 46
Root finder for bvec_f failed on iteration 46
Outer Loop Iteration Number:  47
Root finder for bvec_h failed on iteration 47
Root finder for bvec_f failed on iteration 47
Outer Loop Iteration Number:  48
Root finder for bvec_h failed on iteration 48
Root finder for bvec_f failed on iteration 48
Outer Loop Iteration Number:  49
Root finder for bvec_h failed on iteration 49
Root finder for bvec_f failed on iteration 49
Outer Loop Iteration Number:  50
Root finder for bvec_h failed on iteration 50


/Users/rubyzhang/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:67: RuntimeWarning: overflow encountered in double_scalars
/Users/rubyzhang/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:13: RuntimeWarning: overflow encountered in multiply



Root finder for bvec_h failed on iteration 142
Root finder for bvec_f failed on iteration 142
Outer Loop Iteration Number:  143
Root finder for bvec_h failed on iteration 143
Root finder for bvec_f failed on iteration 143
Outer Loop Iteration Number:  144
Root finder for bvec_h failed on iteration 144
Root finder for bvec_f failed on iteration 144
Outer Loop Iteration Number:  145
Root finder for bvec_h failed on iteration 145
Root finder for bvec_f failed on iteration 145
Outer Loop Iteration Number:  146
Root finder for bvec_h failed on iteration 146
Root finder for bvec_f failed on iteration 146
Outer Loop Iteration Number:  147
Root finder for bvec_h failed on iteration 147
Root finder for bvec_f failed on iteration 147
Outer Loop Iteration Number:  148
Root finder for bvec_h failed on iteration 148
Root finder for bvec_f failed on iteration 148
Outer Loop Iteration Number:  149
Root finder for bvec_h failed on iteration 149
Root finder for bvec_f failed on iteration 149
Outer Loo